# LangChains fake LLm good while developement

In [26]:
%pip install langchain_community python-dotenv openai langchain_anthropic langchain_core langchain_openai

  Using cached langchain_openai-0.3.30-py3-none-any.whl.metadata (2.4 kB)
Using cached langchain_openai-0.3.30-py3-none-any.whl (74 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 999.3/999.3 kB 5.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [langchain_openai]
Note: you may need to restart the kernel to use updated packages.


In [8]:
from langchain_community.llms.fake import FakeListLLM

# Create a fake LLM that always returns the same response
fake_llm = FakeListLLM(responses=["Hello"])
result = fake_llm.invoke("Any input will return Hello") 
print(result) # Output: Hello

Hello


In [18]:
from openai import OpenAI
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Initialize OpenAI client
openAIClient = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

response = openAIClient.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Finantial capital of India?"}]
)
print(response.choices[0].message.content)

The financial capital of India is Mumbai.


## print the reasoning with the response 

### example with claude model

In [ ]:
from langchain_anthropic import ChatAnthropic 
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage


# Create a template
template = ChatPromptTemplate.from_messages([
    ("system", "You are an experienced programmer and mathematicalanalyst."),
    ("user", "{problem}")
])

# Anthropic client
anthropicClient = ChatAnthropic(api_key=os.getenv("ANTHROPIC_API_KEY"),
model_name="claude-3-7-sonnet-20250219", 
thinking={"type": "enabled", "budget_tokens": 10000},
max_tokens=60_000
)

# Create and run a chain
chain = template | anthropicClient

# Complex algorithmic problem
problem = """
Design an algorithm to find the kth largest element in an unsorted array
with the optimal time complexity. Analyze the time and space complexity
of your solution and explain why it's optimal.
"""

response = anthropicClient.invoke([HumanMessage(content=problem)])
print(response.content)


[{'signature': 'ErUBCkYIBhgCIkAWa0xxxdfdjgjqdVdPAGulSMsI8lBxzKzRSiiKx3AU5jnCrJYjR/gi0bLtACvbvvhWEQjcxN/779Qj0kmh3BJtEgy+OmHb//qj8YJ7biQaDEqavPdha7tcwGQ5cCIwvpt5GBXQP5oHkJotdWKbW4enBiQYVkGtHkYOnITA2vLPKdtlqv5K8iE9kNQhYULsKh3B7pR7OIL20bar7mvL4Hlza4dydl+Agw8vJMHbmxgC', 'thinking': "Alright, I'm being asked to design an algorithm to find the kth largest element in an unsorted array, with the optimal time complexity. I also need to analyze the time and space complexity and explain why it's optimal.\n\nFirst, let's explore a few approaches to solve this problem:\n\n1. **Sorting**: We could sort the array and then return the kth largest element. \n   - Time Complexity: O(n log n) where n is the size of the array\n   - Space Complexity: O(1) if we're using an in-place sorting algorithm\n   - This is not optimal in terms of time complexity for this specific problem, because we don't need to sort the entire array.\n\n2. **Min-Heap**: We could use a min-heap of size k.\n   - We iterate through th

# Prompt Template 
 prompt template helps to translate user inputs and parameters to instructions to language model.

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage, SystemMessage

# Example 1: Simple Prompt Template
simple_prompt = PromptTemplate(
    input_variables=["name"],
    template="Hello, {name}! How can I assist you today?"
)
print(simple_prompt.format(name="Alice"))

# Example 2: Chat Prompt Template with System and Human Messages
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "What is the capital of {country}?")
])

formatted_prompt = chat_prompt.format(country="India")
print(formatted_prompt)

# Example 3: Using HumanMessage and SystemMessage explicitly
chat_prompt_explicit = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a knowledgeable assistant."),
    HumanMessage(content="Explain the theory of relativity in simple terms.")
])

print(chat_prompt_explicit)

Hello, Alice! How can I assist you today?
System: You are a helpful assistant.
Human: What is the capital of India?
input_variables=[] input_types={} partial_variables={} messages=[SystemMessage(content='You are a knowledgeable assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Explain the theory of relativity in simple terms.', additional_kwargs={}, response_metadata={})]


# LangChain Expression Language (LCEL) 
Rather than focusing on how to execute each step, LCEL lets you define what you want to accomplish, allowing LangChain to handle the execution details behind the scenes.


Runnable interface is at the core of LCEL, every component built with LCEL adheres to this interface which provides below methods . <br>
invoke() - Processes a single input synchronously and returns an output<br>
stream() - Streams output as it’s being generated <br>
batch() - Efficiently processes multiple inputs in parallel <br>
ainvoke(), abatch(), astream(): Asynchronous versions of the above methods.


## what happens intenally when this chain in invoked??
As each component implements the Runnable Interface all of them have the method invoke().
Each component calls the their invoke() method and pass their output as input to other component.

Flow
Input → [Runnable1.invoke()] → Result1 → [Runnable2.invoke()] → Result2 → [Runnable3.invoke()] → Final Output

In [27]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# Create components
prompt = PromptTemplate.from_template("Tell me a joke about {topic}")
llm = ChatOpenAI()
output_parser = StrOutputParser()

# Chain them together using LCEL
chain = prompt | llm | output_parser

# Use the chain
result = chain.invoke({"topic": "programming"})
print(result)

Why do programmers prefer dark mode? 
Because the light attracts bugs!


### chaining the chain for the complex expressions


In [43]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import os

chat = ChatOpenAI()

# First chain generates a story
story_prompt = PromptTemplate.from_template("Write a short story about {topic}")
story_chain = story_prompt | chat | StrOutputParser()

# Second chain analyzes the story
analysis_prompt = PromptTemplate.from_template("Analyze the following story's mood:\n{story}")
analysis_chain = analysis_prompt | chat | StrOutputParser()


# Combine chains
# story_with_analysis = story_chain | analysis_chain
story_with_analysis = story_chain | analysis_chain

# Run the combined chain
result = story_with_analysis.invoke({"topic": " The Sky"})
# print the result
print(result)

The mood of this story is positive, uplifting, and inspirational. The serene and peaceful setting of the town, combined with Lily's fascination with the sky, creates a sense of wonder and beauty. The appearance of the mysterious figure adds an element of magic and awe to the story, while the task given to Lily to spread love and kindness brings a sense of hope and purpose. As Lily helps to transform the town and bring its people together, the mood becomes one of joy, fulfillment, and gratitude. Overall, the story conveys a sense of goodness and light, leaving the reader with a feeling of warmth and positivity.


In [53]:
# Enhanced output formatting keepig the original story context
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
import os

chat = ChatOpenAI()

# First chain generates a story
story_prompt = PromptTemplate.from_template("Write a short story about {topic}")
story_chain = story_prompt | chat | StrOutputParser()

# Second chain analyzes the story
analysis_prompt = PromptTemplate.from_template("Analyze the following story's mood:\n{story}")
analysis_chain = analysis_prompt | chat | StrOutputParser()


enhanced_chain = RunnablePassthrough.assign(
story=story_chain # Add 'story' key with generated content 
).assign(
analysis=analysis_chain # Add 'analysis' key with analysis of the story
)

# Execute the chain
result = enhanced_chain.invoke({"topic": "a rainy day"})
#print(result.keys())
# print keys and values separately
for key, value in result.items():
    print(f"{key}: {value}")


topic: a rainy day
story: It was a cool, dreary morning as the raindrops tapped against the window pane. Sarah sighed as she looked outside, feeling disappointed that her plans for a day at the park with her friends had been washed away by the relentless downpour. 

As she sat in her cozy living room, the sound of the rain became a comforting background noise. She decided to make the most of the rainy day by curling up on the couch with a good book and a cup of hot tea. The pages of the book fluttered in her hands as she lost herself in a world of fantasy and adventure.

Outside, the rain continued to fall, creating a soothing rhythm that seemed to lull her into a state of relaxation. The soft pitter-patter of the rain on the roof created a peaceful atmosphere, and Sarah found herself feeling grateful for the excuse to slow down and enjoy a quiet day at home.

As the day turned into evening, the rain began to taper off, leaving behind a glistening sheen on the pavement. Sarah stepped o

In [46]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import os

chat = ChatOpenAI()

# First chain generates a story
story_prompt = PromptTemplate.from_template("Do a good research about the concept {topic}")
story_chain = story_prompt | chat | StrOutputParser()

# Second chain analyzes the story
analysis_prompt = PromptTemplate.from_template("Analyze and make it simple to understand for software tech audience give the flow diagram like information :\n{story}")
analysis_chain = analysis_prompt | chat | StrOutputParser()


# Combine chains
# story_with_analysis = story_chain | analysis_chain
story_with_analysis = story_chain | analysis_chain

# Run the combined chain
result = story_with_analysis.invoke({"topic": " The transformer architecture"})
# print the result
#print(result)
import textwrap
print(textwrap.fill(result, width=80))

The Transformer architecture is a deep learning model designed for natural
language processing tasks. It uses self-attention to allow each word in a
sentence to attend to every other word, capturing long-range dependencies. The
model consists of an encoder and a decoder, each with layers of self-attention
and feed-forward neural networks. Its parallelization capabilities make it
faster and more efficient than traditional models like RNNs and LSTMs. The
Transformer architecture has achieved state-of-the-art performance in tasks like
machine translation, text summarization, and question answering, and has been
applied in other domains as well. It represents a significant advancement in
deep learning for sequential data processing and has enabled further innovation
in natural language processing.   Flow diagram: Input (sentence) -> Encoder
(self-attention + feed-forward NN) -> Decoder (self-attention + feed-forward NN)
-> Output (translated sentence)
